# PyMOL Remote demo

### 1. [Prerequisite] Server side (where PyMOL is running)

This demo should be run on the `client` side, i.e. the machine where you want to run your Python code.
To run it, you will need to have run the following command on the `server` side, i.e. the machine where PyMOL is running:

```bash
# Navigate into your pymol environment
#  that environment should have both, pymol and pymol_remote installed
# e.g.:
# conda activate pymol

# If pymol_remote is installed, the following command will start the server
pymol_remote
```

This command will start pymol, and in the pymol console you should see a likely guess of your IP address (and the correct port number, 9123 by default).
If this IP address does not work, you might need to use commands like `ifconfig` or `ipconfig` on your computer to find the correct IP address of the server in your local network.

### 2. Client side (where you want to run your Python code)

In [ ]:
from pymol_remote.client import PymolSession

# NOTE: When you run `pymol_remote` on the server side, it will print a likely guess of your
#  IP address (and the correct port number, 9123 by default). Try that IP address first,
#  if it doesn't work you might need to use commands like `ifconfig` or `ipconfig` on
#  your computer to find the correct IP address of the server in your local network.
IP_ADDRESS_OF_SERVER = (
    "YOUR_SERVERS_IP_ADDRESS_GOES_HERE"  # for example sth. like "10.123.45.67"
)
pymol = PymolSession(hostname=IP_ADDRESS_OF_SERVER, port=9123)

# You can now send commands to PyMOL
pymol.fetch("6lyz")
pymol.do("remove solvent")
pymol.do("set valence, on")
pymol.get_state(format="cif")

In [2]:
pymol.print_help()


    A class for interacting with a PyMOL RPC server, which allows you to execute PyMOL commands
    and retrieve information from a running PyMOL session (possibly running on a remote machine
    on the same network).

    You can find more information about the available commands here:
        - https://pymol.org/pymol-command-ref.html
        - https://pymolwiki.org/index.php/Category:Commands

    You can invoke all pymol commands using direclty as methods on this object.
    For example:

    ```python
    >>> session.fetch('6lyz')
    >>> session.get_names()
    ```

    You may also use any command that can be called in the pymol console with the `do` method.
    For example:

    ```python
    >>> session.do('set valence, on')
    ```

    To get the current state of the pymol session from the server, you can use the `get_state` method.
    For example:

    ```python
    session.get_state(selection="(all)", state=-1, format="cif")
    session.get_state(selection="(all)", state

Get help on a specific command

In [3]:
pymol.help("fetch")

fetch(code, name='', state=0, finish=1, discrete=-1, multiplex=-2, zoom=-1, type='', async_=0, path='', file=None, quiet=1, _self=<module 'pymol.cmd' from '/opt/homebrew/Caskroom/miniconda/base/envs/pymol/lib/python3.12/site-packages/pymol/cmd.py'>, **kwargs)

DESCRIPTION

    "fetch" downloads a file from the internet (if possible)

USAGE

    fetch code [, name [, state [, finish [, discrete [, multiplex
        [, zoom [, type [, async [, path ]]]]]]]]]

ARGUMENTS

    code = a single PDB identifier or a list of identifiers. Supports
    5-letter codes for fetching single chains (like 1a00A).

    name = the object name into which the file should be loaded.

    state = the state number into which the file should loaded.

    type = str: cif, pdb, pdb1, 2fofc, fofc, emd, cid, sid {default: cif
    (default was "pdb" up to 1.7.6)}

    async_ = 0/1: download in the background and do not block the PyMOL
    command line {default: 0 -- changed in PyMOL 2.3}

PYMOL API

    cmd.fetch(st

In [4]:
pymol.help("get_state")

get_state(selection: str = '(all)', state: int = -1, format: str = 'pdb') -> str | bytes
Get the current state of the PyMOL session as a PDB string.

    This function retrieves the current state of the PyMOL session, including all
    molecules, their coordinates, and other relevant information, and returns it
    as a PDB string.

    Args:
        - selection (str, optional): The selection of atoms to include in the PDB string.
            Defaults to "all".
        - state (int, optional): The state of the molecule to include in the PDB string.
            Defaults to `-1`, which means the current state. If state is 0, then
            a multi-state output file is written.
        - format (str, optional): The format of the file to save. Defaults to "pdb".
            Supported formats: "pdb", "mol", "png", "cif"

    Returns:
        str | bytes: The PDB string or binary file content.
    


Show all the available commands

In [5]:
pymol.help()

Get help for a specific command by passing the command name to the `help` method.
For example:
```
session.help('fetch')
```

To get links to more documentation, call `session.print_help()`.
Available commands:

  -abort
  -accept
  -add_bond
  -alias
  -align
  -alignto
  -alphatoall
  -alter
  -alter_list
  -alter_state
  -angle
  -api
  -as_pathstr
  -assign_stereo
  -async_
  -attach
  -auto_measure
  -backward
  -bg_color
  -bg_colour
  -block_flush
  -bond
  -boolean_sc
  -button
  -cache
  -callout
  -capture
  -cartoon
  -cd
  -cealign
  -center
  -centerofmass
  -check
  -clean
  -clip
  -cls
  -color
  -color_deep
  -color_sc
  -colour
  -commands
  -config_mouse
  -contour_sc
  -copy
  -copy_image
  -copy_to
  -count_atoms
  -count_discrete
  -count_frames
  -count_states
  -create
  -curve_new
  -cycle_valence
  -decline
  -del_colorection
  -delete
  -deprotect
  -desaturate
  -deselect
  -dihedral
  -dirty
  -dirty_wizard
  -disable
  -dist
  -distance
  -do
  -download_c